# Introduction

Connecting your components and circuits to the outside world is one of the most recurring tasks in integrated photonics design. Often fiber grating couplers (or edge couples) are used for that purpose. 

Whether you are planning to measure your chip on the optical table or whether you are using standard packaging techniques, it is generally a good practice to neatly place those fiber couplers along a cardinal axis of your chip with a regular distance between them.

Typically, the ports of your circuit won't be as regular. Connecting the irregularly spaced ports of your circuit to the outside world can be a tedious task. 

**With IOFibcouplers this task highly automatized while leaving a large number of degrees of freedom to tweak your connections to the outside world to match your needs.**

Let's get started with the simplest minimal example connecting an MMI to the outside world. First we create the MMI.

In [ ]:
%matplotlib inline
import pylab

In [ ]:
pylab.rcParams['figure.figsize'] = (15, 7)
from technologies.silicon_photonics import TECH
from ipkiss3 import all as i3

from picazzo3.filters.mmi import  MMI1x2Tapered
from picazzo3.traces.wire_wg import WireWaveguideTemplate

# MMI
wg_t1 = WireWaveguideTemplate()
wg_t1.Layout(core_width=7.0, cladding_width=8.0)
# input and output traces of the MMI
wg_t2 = WireWaveguideTemplate()
wg_t2.Layout(core_width=0.6)

my_mmi_1x2 = MMI1x2Tapered(mmi_trace_template=wg_t1,
                          input_trace_template=wg_t2,
                          output_trace_template=wg_t2)

layout=my_mmi_1x2.Layout(length=10.0)
layout.visualize()     

We now connect it to the outside world using IoFibCoup: 

In [ ]:
from picazzo3.container.iofibcoup.cell import IoFibcoup

iofb = IoFibcoup(contents=my_mmi_1x2) 
iofb_layout = iofb.Layout() 

iofb_layout.visualize()
iofb_layout.write_gdsii("iofibcoup_simple1.gds")

You can see that 3 fiber grating couplers have been created. The the southmost fibercoupler on the west side is placed in the origin and the southmost fibercoupler on the east side is placed in south_east a property of the layoutview. Of course there are many other properties that control many aspects how the fiber couplers are placed. Depending on the desired level of control, specific fibercoupler classes should be used:


* **``IoFibcoup``** is the simplest one, using the same grating coupler for all ports, and uniform settings for routing the ports.
* **``IoFibcoupEastWest``** offers the possibility to differentiate the East and West settings, so you can for instance use a different 
grating coupler on both sides
* **``IoFibcoupGeneric``** allows you to customize every individual grating coupler, its transformation, and the trace template for the 
routes used to connect the component to the grating.

In what follows we will introduce those three classes - starting with the simplest one : IoFibcoup 

## IoFibcoup

**IOFibcoup** is the most general IO-Fibercoupler and best used when the properties specifying the connection to the ouside world are 
**uniform** among all ports. 

Each connection built from several parts. 

* A **Component section**: Place where the circuit that has to be connected to the ouside world is placed. If needed tapers added to fit the  trace template used in the Fanout section.
* **A Fanout section**: On the east and west side of your component the ports are fanned out to a pitch set by ``y_spacing``.
* A **Connect section**: After the Fanout a straight waveguides are used to reach the fiber grating couplers. 
* A **Grating Coupler section**: There the grating couplers are placed also with a pitch of ``y_spacing``.

![IOFibCoup](_images/IOFibCoupZones.png)

Let's give it a try. You can also control the ``y_spacing`` and ``contents_transformation`` to apply a transformation to the contents just as is the case with any container. To have lower losses in the connect section we use a trace template with a wider core.

In [ ]:
# connect_template: wider to reduce losses
wgc_t = WireWaveguideTemplate()
wgc_t.Layout(core_width=2.0)

iofb = IoFibcoup(contents=my_mmi_1x2, connect_trace_template=wgc_t) 
iofb_layout = iofb.Layout(y_spacing = 70,
                          contents_transformation=i3.Rotation(rotation=-20)) 
iofb_layout.visualize()
iofb_layout.write_gdsii("iofibcoup_simple2.gds")

Of course there are may other parameters you can control on the layout level. Most of the parameters deal with the length of each section and the length of the transitions between sections. The figure below illustrate those parameters.

![IOFibCoup](_images/IOFibCoup.png)

With the interact below you can play with some of them.

In [ ]:
def create_fibIO(rotation=10.0,
                 y_spacing=50.0,
                 south_east_x=500.0,
                 absolute_offset_x=0.0,
                 absolute_offset_y=0.0, 
                 fanout_length = 100,                                                                  
                 connect_transition_length=20.0 ,                       
                 fiber_coupler_transition_length=30.0):
    
    iofb = IoFibcoup(contents=my_mmi_1x2,
                     connect_trace_template=wgc_t)


    iofb_layout = iofb.Layout(contents_transformation=i3.Rotation(rotation=rotation),  # rotate the contents
                              y_spacing = y_spacing,                                   # y_spacing between the ports. 
                              south_west=(0.0, 0.0),                                   # south west corner of the adapter
                              south_east=(south_east_x, 0.0),                          # south east corner of the adapter
                              absolute_offset=(absolute_offset_x, absolute_offset_y),                    # offset of component from the center
                              fanout_length = fanout_length,                           # Length of the fanout                               
                              connect_transition_length=connect_transition_length,     # transition length
                              fiber_coupler_transition_length=fiber_coupler_transition_length,# transition length between connection WG and the fiber coupler
                              bend_radius = 10.0                                       # bend radius used in all bends.  
                              )
    
    iofb_layout.visualize()

    
from IPython.html.widgets import interact 
interact(create_fibIO,
         rotation=(-44,44,1),
         y_spacing=(20,150,10),
         south_east_x= (200,2000,100),
         absolute_offset_x=(-100,100,10),
         absolute_offset_y=(-100,100,10), 
         fanout_length = (20,500,50),                                                                  
         connect_transition_length=(10,100.0,10),                       
         fiber_coupler_transition_length=(10,100.0,10))

<h4>Positioning of the contents</h4>

As shown above, using the ``absolute_offset`` parameter, the offset of the component from the (0,0) of the IoFibcoup cell can be specified. 

if you do not specify this offset (or specify None), an offset will be calculated such that the west and east ports of the contents are X-aligned around the center of the IoFibcoup, and the southmost West port of the contents will be aligned to the southwest. A relative offset which will be added to this absolute_offset can be specified using the ``relative_offset`` parameter.

<img src="_images/IOFibCoupRelative.png" width=1000 >

With default position:

In [ ]:
iofb = IoFibcoup(contents=my_mmi_1x2,
                 connect_trace_template=wgc_t)


iofb_layout = iofb.Layout(y_spacing = 50.0     ,                               # y_spacing between the ports. 
                          south_west=(0.0, 0.0),                               # south west corner of the adapter
                          south_east=(500, 0.0),                               # south east corner of the adapter
                          fanout_length = 100.0,                               # Length of the fanout                               
                          connect_transition_length=20.0,                      # transition length
                          fiber_coupler_transition_length=30.0,                # transition length between connection WG and the fiber coupler
                          bend_radius = 10.0                                   # bend radius used in all bends.  
                          )
    
iofb_layout.visualize()

With ``relative_offset``:

In [ ]:
iofb = IoFibcoup(contents=my_mmi_1x2,
                 connect_trace_template=wgc_t)


iofb_layout = iofb.Layout(y_spacing = 50.0     ,                               # y_spacing between the ports. 
                          south_west=(0.0, 0.0),                               # south west corner of the adapter
                          south_east=(500, 0.0),                               # south east corner of the adapter
                          fanout_length = 100.0,                               # Length of the fanout                               
                          connect_transition_length=20.0,                      # transition length
                          fiber_coupler_transition_length=30.0,                # transition length between connection WG and the fiber coupler
                          bend_radius = 10.0,                                   # bend radius used in all bends.  
                          relative_offset = (-50.0, 30.0)                     # relative position
                          )
    
iofb_layout.visualize()

<h4> Modifying grating couplers and trace templates </h4>

Of course you can also modify the grating couplers and other trace templates used in the IOFibCoup class.

In [ ]:
# normal waveguide template: single mode
wg_t = WireWaveguideTemplate()
wg_t.Layout(core_width=1.0)

# connect_template: wider to reduce losses
wgc_t = WireWaveguideTemplate()
wgc_t.Layout(core_width=2.0)


iofb = IoFibcoup(contents=my_mmi_1x2,
                 trace_template = wg_t, #Default trace template when none are specified.
                 connect_trace_template=wgc_t, #Trace template for the connect section
                 fiber_coupler=i3.TECH.IO.FIBCOUP.CURVED.PCELLS.DEFAULT_GRATING) 

iofb_layout = iofb.Layout(south_east=(500.0, 0.0),
                          y_spacing = 70,
                          contents_transformation=i3.Rotation(rotation=-20)) 
iofb_layout.visualize()
iofb_layout.write_gdsii("iofibcoup_simple3.gds")

<h4>Explicitly setting the ports to use</h4>

Sometimes because of the application of a transformation there are no ports facing east or west.

In [ ]:
iofb = IoFibcoup(contents=my_mmi_1x2,
                 trace_template = wg_t, #Default trace template when none are specified.
                 fiber_coupler=i3.TECH.IO.FIBCOUP.CURVED.PCELLS.DEFAULT_GRATING) 

iofb_layout = iofb.Layout(south_east=(500.0, 0.0),
                          y_spacing = 70,
                          contents_transformation=i3.Rotation(rotation=-80)) 
iofb_layout.visualize()

In that case or if you want, you can explicitely set which ports need to be ported east and which need to be ported west. Make sure that they are ordered correctly. The first port in the list is placed first on the south side.

In [ ]:
iofb = IoFibcoup(contents=my_mmi_1x2,
                 trace_template = wg_t, #Default trace template when none are specified.
                 west_port_labels = [], 
                 east_port_labels = ['out1', 'out2', 'in'],
                 fiber_coupler=i3.TECH.IO.FIBCOUP.CURVED.PCELLS.DEFAULT_GRATING) 

iofb_layout = iofb.Layout(south_east=(500.0, 0.0),
                          y_spacing = 70,
                          contents_transformation=i3.Rotation(rotation=-20)) 
iofb_layout.visualize()
iofb_layout.write_gdsii("iofibcoup_simple4.gds")

## CircuitModels of IOFibcoup

IOFibcoup also builds up a hierarchical CircuitModel of all the components inside it. This way you can easily make air to air simulations of your experimental setup. Here we will use IOFibCoup to connect a ring to the outside world. Let's first get a look a the layout.

In [ ]:
import numpy as np
import pylab as plt

# Waveguides
wg_t = WireWaveguideTemplate()
wg_t.Layout(core_width=0.45)

#Ring Resonator
from picazzo3.filters.ring import RingRect180DropFilter
my_ring = RingRect180DropFilter()

# Curved grating
from picazzo3.fibcoup.curved import FiberCouplerCurvedGrating
grating = FiberCouplerCurvedGrating()

from picazzo3.container.iofibcoup import IoFibcoup
iofb = IoFibcoup(contents=my_ring,
                 fiber_coupler=grating,
                 trace_template=wg_t)

layout = iofb.Layout(south_east=(500.0, 0.0), y_spacing=60.0) 
layout.visualize()

To make a caphe simulation we first have to set the caphemodels of our ring resonator, the grating coupler and waveguides.

In [ ]:
# Setting the Caphemodel from the ring. 
cp = dict(cross_coupling1=1j*0.05**0.5,
          straight_coupling1=0.92**0.5,
          reflection_in1=1j*0.05,
          )

my_ring_cm = my_ring.CircuitModel(ring_length=40.0,             # we can manually specify the ring length
                                    coupler_parameters=[cp, cp]) # 2 couplers

# Setting the Caphemodel from the grating coupler.
gm = grating.CircuitModel(center_wavelength=1.55, bandwidth_3dB=0.1, peak_transmission=0.5, reflection=0.1)


In [ ]:

wavelengths =  np.linspace(1.50, 1.6, 2001)

# Note: Term naming might change in the future...
R = my_ring_cm.get_smatrix(wavelengths=wavelengths)
plt.plot(wavelengths, np.abs(R['in1', 'out1'])**2, 'b', label="pass")
plt.plot(wavelengths, np.abs(R['in1', 'out2'])**2, 'r', label="drop")
plt.xlabel("Wavelength ($\mu m$)")
plt.ylabel("Power transmission")
plt.legend()
plt.show()   

And we can now start to simulate the entire IOFibCoup component.

In [ ]:
# Using the model from the IOColumn from an coupler to coupler simulation. 
cm = iofb.CircuitModel()

my_engine = i3.CapheFrequencyEngine()
wavelengths =  np.linspace(1.50, 1.6, 2001)
my_simulation = my_engine.SMatrixSimulation(model=cm,
                                            wavelengths=wavelengths)
my_simulation.run()

# Note: Term naming will change in the future...
R = cm.get_smatrix(wavelengths=wavelengths)
plt.plot(wavelengths, np.abs(R['in1', 'out1'])**2, 'b', label="pass")
plt.plot(wavelengths, np.abs(R['in1', 'out2'])**2, 'r', label="drop")
plt.plot(wavelengths, np.abs(R['in1', 'in2'])**2, 'g', label="add")
plt.xlabel("Wavelength ($\mu m$)")
plt.ylabel("Power transmission")
plt.legend()
plt.show()   

We can clearly see the ripple of the the reflections in the grating and in the couplers! You can play with the parameters and see when ripple might be a problem for you or try to reproduce qualitative results you may have experienced in your experiments. 

## IoFibcoupEastWest

**IoFibcoupEastWest**  is practical when you want to use different connections to the east as to the west. **IoFibcoupEastWest** has exactly the same properties as **IoFibcoup** except that all the properties that have an influence on the connections to the outside world our now doubled. One refering to the **east** and one refering to the **west**. 

The example below should be rather self-explantory.   

In [ ]:
from picazzo3.container.iofibcoup import IoFibcoupEastWest

iofb = IoFibcoupEastWest(contents=my_mmi_1x2,                       # the component
                        east_trace_template=wg_t,           # trace template for East fanout
                        west_trace_template=wg_t,           # trace_template for West fanout
                        east_connect_trace_template=wgc_t,  # trace template for wide connections
                        west_connect_trace_template=wg_t,   # trace template for wide connections
                        east_fiber_coupler=i3.TECH.IO.FIBCOUP.CURVED.PCELLS.DEFAULT_GRATING,             # fiber coupler East
                        west_fiber_coupler=i3.TECH.IO.FIBCOUP.STRAIGHT.PCELLS.DEFAULT_GRATING)           # fiber coupler West

iofb_layout = iofb.Layout(contents_transformation=i3.Rotation(rotation=-30.0),  # rotate the contents
                          east_connect_transition_length=30.0,                  # transition length
                          east_fiber_coupler_transition_length=50.0,            # transition length between connection WG and the fiber coupler
                          west_fiber_coupler_transition_length=20.0,            # transition length between connection WG and the fiber coupler
                          south_west=(-0.0, 0.0),                               # south west corner of the adapter
                          south_east=(500.0, 0.0),                              # south east corner of the adapter
                          relative_offset=(-100.0, 70.0)                        # offset of component from it's default position
                          )        
iofb_layout.visualize()
iofb_layout.write_gdsii("iofibcoup_simple5.gds")

## IoFibcoupGeneric

**IoFibcoupGeneric** is required if you want to control each connection separately. All the properties that relate to a connection to the ouside world are "plural". This means that they can accept a list of parameters (e.g. ``east_trace_templates``).

This allows you to specify the template (or other parameters) for each individual port. The lists do not need to have the exact 
same length as the number of ports. If the number of elements is different than the number of ports in your component,
the ``IoFibcoupGeneric`` will just cycle through the list. That way, if all the ports use the same value, you only need to supply
a list of one element.

Lets explain this using an example:

First we make an MMI that has 3 inputs and 5 outputs.

In [ ]:
from picazzo3.filters.mmi import MMIIdenticalTapered
wg_t1 = WireWaveguideTemplate()
wg_t1.Layout(core_width=7.0, cladding_width=8.0)
# input and output traces of the MMI
wg_t2 = WireWaveguideTemplate()
wg_t2.Layout(core_width=0.8)

my_mmi = MMIIdenticalTapered(mmi_trace_template=wg_t1,
                             input_trace_template=wg_t2,
                             output_trace_template=wg_t2,
                             trace_template=i3.TECH.PCELLS.WG.DEFAULT,
                             n_inputs=3,
                             n_outputs=5)

layout = my_mmi.Layout(length=10.0, 
                  input_y_positions=[-2.0, 0.0, 2.0], 
                  output_y_positions=[-2.0, -1.0, 0.0, 1.0, 2.0]
                      )

layout.visualize()

We also need a couple of trace templates and grating couplers.

In [ ]:
# Grating Couplers
my_gc_1 = i3.TECH.IO.FIBCOUP.STRAIGHT.PCELLS.DEFAULT_GRATING
my_gc_2 = i3.TECH.IO.FIBCOUP.CURVED.PCELLS.DEFAULT_GRATING
# Trace templates
wg_t1 = WireWaveguideTemplate()
wg_t1.Layout(core_width=5.0, cladding_width=8.0)

wg_t2 = WireWaveguideTemplate()
wg_t2.Layout(core_width=2.0, cladding_width=8.0)

wg_t3 = WireWaveguideTemplate()

Lets now create the generic IOFibCoup:

In [ ]:
# making the adapter
from picazzo3.container.iofibcoup import IoFibcoupGeneric
iofb = IoFibcoupGeneric(contents=my_mmi,                       # the component
                        east_trace_templates=[wg_t3],           # trace template for East fanout
                        west_trace_templates=[wg_t3],           # trace_template for West fanout
                        east_connect_trace_templates=[wg_t1,wg_t2],  # trace template for wide connections
                        west_connect_trace_templates=[wg_t1],  # trace template for wide connections
                        east_fiber_couplers=[my_gc_1, my_gc_2],           # fiber couplers East
                        west_fiber_couplers=[my_gc_2])           # fiber couplers West

iofb_layout = iofb.Layout(east_connect_transition_lengths=[30.0],              # transition lengths
                          east_fiber_coupler_transformations=[i3.HMirror(), i3.HMirror() + i3.Translation((-30.0, -5.0))],
                          east_fiber_coupler_transition_lengths=[50.0, 100.0, 25.0], # transition lengths between connection WG and the fiber coupler
                          south_west=(-0.0, 50.0),             # south west corner of the adapter
                          south_east=(1000.0, 100.0),           # south east corner of the adapter
                          y_spacing = 50.0
                          )

iofb_layout.visualize()
iofb_layout.write_gdsii("iofibcoup_simple6.gds")

Note how the cyclic list work: for instance on the east side we have used ``east_fiber_couplers=[my_gc_1, my_gc_2]``. This resulted in the alternation of ``my_gc_1`` and ``my_gc_2`` starting with ``my_gc_1``. This is then independent of the number of ports on the east side, which can be quite practical.

Also note the use of ``east_fiber_coupler_transformations``. This allows you to introduce some offset between the different tapers. 

Congratulations! You have now learned most of what can be learned about IOFibCoup's. As an exercise you can try to use it with a real circuit - not just one component.